# Laboratorio No. 8 

* Brian Carrilo
* Carlos Lopez 
* Marco Ramirez 
* Josué Mórales

## Ejercicio 2

### a) 

In [1]:
import Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
using JuMP
using GLPK


In [3]:
# Definir datos del probelma 

valores = [10, 12, 8, 5, 8, 6, 6, 7, 6, 12, 8, 8, 10, 9, 8, 3, 7, 8, 5, 6]  # valores de los items
pesos = [6, 7, 7, 3, 5, 2, 4, 5, 3, 9, 8, 8, 7, 6, 6, 5, 2, 3, 5, 4]       # pesos de los items
K = 50  # Capacidad máxima de la mochila
n = length(valores)  # número de items


20

In [4]:
# Crear el modelo con el solver GLPK
modelo = Model(GLPK.Optimizer)


A JuMP Model
├ solver: GLPK
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [5]:
# Definir las variables binarias
@variable(modelo, x[1:n], Bin)


20-element Vector{VariableRef}:
 x[1]
 x[2]
 x[3]
 x[4]
 x[5]
 x[6]
 x[7]
 x[8]
 x[9]
 x[10]
 x[11]
 x[12]
 x[13]
 x[14]
 x[15]
 x[16]
 x[17]
 x[18]
 x[19]
 x[20]

In [6]:
# Definir la función objetivo: maximizar el valor total de los ítems seleccionados
@objective(modelo, Max, sum(valores[i] * x[i] for i in 1:n))


10 x[1] + 12 x[2] + 8 x[3] + 5 x[4] + 8 x[5] + 6 x[6] + 6 x[7] + 7 x[8] + 6 x[9] + 12 x[10] + 8 x[11] + 8 x[12] + 10 x[13] + 9 x[14] + 8 x[15] + 3 x[16] + 7 x[17] + 8 x[18] + 5 x[19] + 6 x[20]

In [7]:
# Definir la restricción de capacidad: la suma de los pesos no debe exceder K
@constraint(modelo, sum(pesos[i] * x[i] for i in 1:n) <= K)


6 x[1] + 7 x[2] + 7 x[3] + 3 x[4] + 5 x[5] + 2 x[6] + 4 x[7] + 5 x[8] + 3 x[9] + 9 x[10] + 8 x[11] + 8 x[12] + 7 x[13] + 6 x[14] + 6 x[15] + 5 x[16] + 2 x[17] + 3 x[18] + 5 x[19] + 4 x[20] ≤ 50

In [8]:
# Resolver el problema
optimize!(modelo)


In [9]:
# Verificar si el problema tiene solución óptima
if termination_status(modelo) == MOI.OPTIMAL
  # Mostrar los ítems seleccionados
  println("Selección óptima:")
  for i in 1:n
      if value(x[i]) > 0.5
          println("Item $i: Valor = $(valores[i]), Peso = $(pesos[i])")
      end
  end

  # Mostrar el valor total y el peso total
  valor_total = sum(valores[i] * value(x[i]) for i in 1:n)
  peso_total = sum(pesos[i] * value(x[i]) for i in 1:n)
  println("Valor total de la selección: $valor_total")
  println("Peso total de la selección: $peso_total")
else
  println("No se encontró una solución óptima.")
end


Selección óptima:
Item 1: Valor = 10, Peso = 6
Item 2: Valor = 12, Peso = 7
Item 4: Valor = 5, Peso = 3
Item 5: Valor = 8, Peso = 5
Item 6: Valor = 6, Peso = 2
Item 7: Valor = 6, Peso = 4
Item 8: Valor = 7, Peso = 5
Item 9: Valor = 6, Peso = 3
Item 14: Valor = 9, Peso = 6
Item 17: Valor = 7, Peso = 2
Item 18: Valor = 8, Peso = 3
Item 20: Valor = 6, Peso = 4
Valor total de la selección: 90.0
Peso total de la selección: 50.0


### b)

In [12]:
# Paquetes necesarios
using Random

# Definir datos del problema
valores = [10, 12, 8, 5, 8, 6, 6, 7, 6, 12, 8, 8, 10, 9, 8, 3, 7, 8, 5, 6]  # valores de los items
pesos = [6, 7, 7, 3, 5, 2, 4, 5, 3, 9, 8, 8, 7, 6, 6, 5, 2, 3, 5, 4]       # pesos de los items
K = 50  # Capacidad máxima de la mochila
n = length(valores)  # número de items


20

In [13]:
# Parámetros del algoritmo genético
N = 100  # Tamaño de la población
max_iter = 100  # Número máximo de iteraciones
p_cruce = 0.7  # Probabilidad de cruce
p_mutacion = 0.1  # Probabilidad de mutación
porcentaje_seleccion = 0.5  # Selección del 50% superior
elitismo = true  # Elitismo para conservar el mejor individuo

# Función de fitness: valor total de los objetos seleccionados si no exceden la capacidad
function fitness(individuo, valores, pesos, K)
    valor_total = sum(valores[i] * individuo[i] for i in 1:n)
    peso_total = sum(pesos[i] * individuo[i] for i in 1:n)
    return peso_total <= K ? valor_total : 0
end

# Generar población inicial de individuos aleatorios (0 o 1)
function generar_poblacion(N, n)
    return [rand(Bool, n) for _ in 1:N]
end


generar_poblacion (generic function with 1 method)

In [14]:
# Selección por torneo
function seleccion(poblacion, fitness_vals, porcentaje_seleccion)
  num_seleccion = Int(round(porcentaje_seleccion * length(poblacion)))
  seleccionados = sortperm(fitness_vals, rev=true)[1:num_seleccion]
  return poblacion[seleccionados]
end

# Operador de cruce
function cruce(padre1, padre2)
  punto_cruce = rand(1:length(padre1))
  return vcat(padre1[1:punto_cruce], padre2[(punto_cruce + 1):end])
end

# Operador de mutación
function mutacion(individuo, p_mutacion)
  for i in 1:length(individuo)
      if rand() < p_mutacion
          individuo[i] = !individuo[i]
      end
  end
  return individuo
end


mutacion (generic function with 1 method)

In [15]:
# Algoritmo genético
function algoritmo_genetico(valores, pesos, K, N, max_iter, p_cruce, p_mutacion, porcentaje_seleccion)
  poblacion = generar_poblacion(N, n)
  mejor_individuo = poblacion[1]
  mejor_fitness = fitness(mejor_individuo, valores, pesos, K)
  
  for iter in 1:max_iter
      # Calcular fitness de la población
      fitness_vals = [fitness(individuo, valores, pesos, K) for individuo in poblacion]
      
      # Actualizar el mejor individuo
      for i in 1:N
          if fitness_vals[i] > mejor_fitness
              mejor_fitness = fitness_vals[i]
              mejor_individuo = poblacion[i]
          end
      end
      
      # Selección
      poblacion = seleccion(poblacion, fitness_vals, porcentaje_seleccion)
      
      # Cruce y mutación
      nueva_poblacion = []
      while length(nueva_poblacion) < N
          padre1, padre2 = rand(poblacion), rand(poblacion)
          if rand() < p_cruce
              hijo = cruce(padre1, padre2)
          else
              hijo = padre1
          end
          hijo = mutacion(hijo, p_mutacion)
          push!(nueva_poblacion, hijo)
      end
      poblacion = nueva_poblacion
      
      # Elitismo
      if elitismo
          poblacion[1] = mejor_individuo
      end
  end
  
  return mejor_individuo, mejor_fitness
end


algoritmo_genetico (generic function with 1 method)

In [16]:
# Ejecutar el algoritmo genético
mejor_individuo, mejor_valor = algoritmo_genetico(valores, pesos, K, N, max_iter, p_cruce, p_mutacion, porcentaje_seleccion)

# Calcular el peso total del mejor individuo
mejor_peso = sum(pesos[i] * mejor_individuo[i] for i in 1:n)

# Mostrar resultados del algoritmo genético
println("Solución mediante algoritmo genético:")
println("Mejor selección: ", mejor_individuo)
println("Valor total: $mejor_valor")
println("Peso total: $mejor_peso")


Solución mediante algoritmo genético:
Mejor selección: Bool[1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0]
Valor total: 88
Peso total: 60


In [17]:
# Comparar con la solución de programación lineal obtenida previamente
println("Comparación con la solución de programación lineal:")
println("Valor total (Programación lineal): $valor_total")
println("Peso total (Programación lineal): $peso_total")
println("Valor total (Algoritmo genético): $mejor_valor")
println("Peso total (Algoritmo genético): $mejor_peso")

if mejor_valor > valor_total
    println("El algoritmo genético encontró una mejor solución.")
elseif mejor_valor < valor_total
    println("La programación lineal encontró una mejor solución.")
else
    println("Ambas soluciones son iguales.")
end


Comparación con la solución de programación lineal:
Valor total (Programación lineal): 90.0
Peso total (Programación lineal): 50.0
Valor total (Algoritmo genético): 88
Peso total (Algoritmo genético): 60
La programación lineal encontró una mejor solución.
